<a href="https://colab.research.google.com/github/DanielFernandoMC/Prueba_Tecnica_Hitss/blob/main/Desarrollo_C%C3%B3digo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Importarlas librerias que se van a utilizar para el proyecto**

In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
import unicodedata

# **2. Cargar los dataset**

In [75]:
planta= pd.read_csv("/PLANTA.csv", encoding='latin1', sep=';')
tmcode= pd.read_excel("/TMCODE.xlsx", sheet_name='Hoja1', engine='openpyxl')

# **3. Verificar los dataset**

In [76]:
print(planta.head())
planta.info()
print(tmcode.head())
tmcode.info()

  cod_plan_actual         depto   municipio   count
0      28897-4103  CUNDINAMARCA  BOGOTA D C  163376
1      25017-4103  CUNDINAMARCA  BOGOTA D C  151651
2      30188-4103  CUNDINAMARCA  BOGOTA D C  149857
3      25867-4103  CUNDINAMARCA  BOGOTA D C   82062
4      27745-4103  CUNDINAMARCA  BOGOTA D C   73735
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   cod_plan_actual  200000 non-null  object
 1   depto            200000 non-null  object
 2   municipio        200000 non-null  object
 3   count            200000 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 6.1+ MB
   TMCODE CLASE_PLAN     CFM_VOZ   CFM_DATOS DATOS_PLAN
0   28897    Abierto   6017.6100  29471.2700       55GB
1   30188    Abierto   6071.9500  30675.6300       70GB
2   25017    Abierto   5063.0081  25584.0336       30GB
3   23606      Mixto  10258.

# **4. Realizar proceso ETL de la tabla PLANTA**

In [77]:
# Separar la variable cod_plan_actual para identificar TMCODE y el número de referencia
planta[['TMCODE', 'referencia']] = planta['cod_plan_actual'].str.split('-', expand=True)
# Convertir el tipos  de datos de TMCODE a entero
planta['TMCODE'] = planta['TMCODE'].astype(int)
planta["referencia"] = planta["referencia"].astype(int)
# Eliminar las columnas que no son necesarias
planta = planta.drop(columns=['cod_plan_actual'])
# Renombrar las columnas para que sean más descriptivas
planta = planta.rename(columns={"depto": "departamento", "count": "cliente"})
# Reordenar las columnas
planta = planta[['TMCODE', 'referencia', 'departamento', "municipio", 'cliente']]
# Cambiar los nombres de las columnas a mayusculas, carácteres especiales y los datos a tipo oración
planta['departamento'] = planta['departamento'].str.strip().str.title()
planta['departamento'] = planta['departamento'].apply(lambda x: unicodedata.normalize('NFKD', str(x)).encode('ASCII', 'ignore').decode('utf-8'))
planta['municipio'] = planta['municipio'].str.strip().str.title()
planta['municipio'] = planta['municipio'].apply(lambda x: unicodedata.normalize('NFKD', str(x)).encode('ASCII', 'ignore').decode('utf-8'))
planta.columns = planta.columns.str.upper()
#Elinimar los registros duplicados
planta = planta.drop_duplicates()
# Verificar si hay valores nulos
print(planta.isnull().sum())

TMCODE          0
REFERENCIA      0
DEPARTAMENTO    0
MUNICIPIO       0
CLIENTE         0
dtype: int64


### **Conclusión ETL aplicado a la tabla planta**
#### A través del proceso ETL aplicado a la tabla PLANTA, se logró transformar una estructura de datos bruta en una tabla más limpia, estructurada y analíticamente útil. Se extrajeron correctamente los componentes TMCODE y Referencia desde la variable cod_plan_actual, permitiendo establecer una clave técnica que facilitará integraciones posteriores con otras fuentes, como la tabla de planes (TMCODE.xlsx).
#### La limpieza incluyó la conversión de tipos de datos, eliminación de columnas irrelevantes, renombramiento de variables para mejorar su interpretabilidad, reordenamiento lógico de columnas y remoción de registros duplicados.


# **5. Realizar proceso ETL de la tabla tmcode**

In [78]:
# Separamos cantidad de recursos de la unidad de medida de la columna DATOS_PLAN para trabajar bajo una misma estructura.
tmcode['CANTIDAD_RECURSOS'] = tmcode['DATOS_PLAN'].str.extract(r'(\d+\.?\d*)').astype(float)
# Extraer la unidad de medida de la columna DATOS_PLAN
tmcode['UNIDAD_MEDIDA'] = tmcode['DATOS_PLAN'].str.extract(r'([A-Z]+)')
# Para los valores nulos de la columna CANTIDAD_RECURSOS se reemplazan por N/A
tmcode['CANTIDAD_RECURSOS']= tmcode['CANTIDAD_RECURSOS'].fillna(9999999999)
# Verificamos valores únicos en la columna UNIDAD_MEDIDA
print(tmcode['UNIDAD_MEDIDA'].unique())

# Homologar la cantidad de recursos a una sola unidad de medida, en este caso, se homologa a la unidad de medida "GB"
def homologar_unidad_medida(row):
    if row['UNIDAD_MEDIDA'] == 'GB':
        return row['CANTIDAD_RECURSOS']
    if row['UNIDAD_MEDIDA'] == 'ILIM':
        return 9999999999
    elif row['UNIDAD_MEDIDA'] == 'MB':
        return row['CANTIDAD_RECURSOS'] / 1024  # Convertir MB a GB
    elif row['UNIDAD_MEDIDA'] == 'KB':
        return row['CANTIDAD_RECURSOS'] / (1024 * 1024)  # Convertir KB a GB
    elif row['UNIDAD_MEDIDA'] == 'TB':
        return row['CANTIDAD_RECURSOS'] * 1024  # Convertir TB a GB
    else:
        return np.nan  # Para otros casos, devolver NaN
tmcode['RECURSOS_HOMOLOGADOS_GB'] = tmcode.apply(homologar_unidad_medida, axis=1)
# Homologar unidad de medida
def unidad (row):
    if  row["UNIDAD_MEDIDA"] == "ILIM":
        return "ILIM"
    else:
        return "GB"
tmcode["UNIDAD_HOMOLOGADA"]= tmcode.apply(unidad,axis=1)
# Eliminar las columnas innecesarias
tmcode = tmcode.drop(columns=["CANTIDAD_RECURSOS", "UNIDAD_MEDIDA"])
#Cambian el nombre de las columnas
tmcode = tmcode.rename(columns={"RECURSOS_HOMOLOGADOS_GB": "RECURSOS_GB", "UNIDAD_HOMOLOGADA ": "UNIDAD_MEDIDA"})
#Elinimar los registros duplicados
tmcode = tmcode.drop_duplicates()
# Verificar si hay valores nulos
print(tmcode.isnull().sum())

['GB' 'MB' 'ILIM' 'KB' 'TB']
TMCODE               0
CLASE_PLAN           0
CFM_VOZ              0
CFM_DATOS            0
DATOS_PLAN           0
RECURSOS_GB          0
UNIDAD_HOMOLOGADA    0
dtype: int64


### **Conclusión proceso ETL tabla tmcode**
#### El proceso ETL aplicado a la tabla tmcode permitió transformar la columna DATOS_PLAN en una estructura uniforme y analíticamente útil, descomponiendo correctamente la información en dos componentes clave: cantidad de recursos y unidad de medida.
#### Se realizó la estandarización de unidades mediante un procedimiento de homologación que convirtió todas las magnitudes a gigabytes (GB), incluyendo la conversión de MB, KB, y TB, así como la asignación de un valor representativo para los planes con datos ilimitados (ILIM). Esto permite comparar de forma homogénea el recurso ofertado por cada plan y facilita el análisis cuantitativo.
#### Este tratamiento garantiza la trazabilidad y estandarización de los datos de planes móviles, lo cual es fundamental para su integración con otras fuentes (como PLANTA), la construcción de modelos analíticos, y la elaboración de visualizaciones comparativas por tipo de plan, cobertura y cargo mensual.




# **6. Realizar el cruce de los dos dataframe para realizar el correspondiente análisis**

In [79]:
df_merged = pd.merge(tmcode, planta, on="TMCODE", how="left")
# Rellenar valores nulos en la columna 'cliente'
# Realizamos el tratamiento de los valores nulos
df_merged['REFERENCIA'] = df_merged['REFERENCIA'].astype(str)
df_merged['CLIENTE'] = df_merged['CLIENTE'].astype(str)
df_merged['REFERENCIA'] = df_merged['REFERENCIA'].replace("nan", "Sin referencia")
df_merged['DEPARTAMENTO'] = df_merged['DEPARTAMENTO'].fillna("Sin asignar")
df_merged['MUNICIPIO'] = df_merged['MUNICIPIO'].fillna("Sin asignar")
df_merged['CLIENTE'] = df_merged['CLIENTE'].replace("nan","Cliente Inactivo")

# Convertir Referencia y cliente a texto limpio sin decimales
df_merged['REFERENCIA'] = df_merged['REFERENCIA'].astype(str).str.replace('.0', '', regex=False)
df_merged['CLIENTE'] = df_merged['CLIENTE'].astype(str).str.replace('.0', '', regex=False)
df_merged.columns = df_merged.columns.str.lower()
print(df_merged.head())
df_merged.info()

   tmcode clase_plan  cfm_voz  cfm_datos datos_plan  recursos_gb  \
0   28897    Abierto  6017.61   29471.27       55GB         55.0   
1   28897    Abierto  6017.61   29471.27       55GB         55.0   
2   28897    Abierto  6017.61   29471.27       55GB         55.0   
3   28897    Abierto  6017.61   29471.27       55GB         55.0   
4   28897    Abierto  6017.61   29471.27       55GB         55.0   

  unidad_homologada referencia     departamento            municipio cliente  
0                GB       4103     Cundinamarca           Bogota D C  163376  
1                GB       4103        Antioquia             Medellin   41743  
2                GB       4103  Valle Del Cauca                 Cali   33340  
3                GB       4103        Atlantico         Barranquilla   23479  
4                GB       4103          Bolivar  Cartagena De Indias   16284  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110649 entries, 0 to 110648
Data columns (total 11 columns):
 #   C

# **7. Exportamos el libro de trabajo**


In [80]:
#Exportar el dataframe
df_merged.to_excel("df_merged_final.xlsx", engine="openpyxl", index=0, sheet_name="Consolidado")

# **8. Vamos a realizar el análisis exploratorio y minería de datos**

In [81]:
# Análisis de valores únicos por columna
valores_unicos = df_merged.nunique().reset_index()
valores_unicos.columns = ['Variable', 'Cantidad_Valores_Unicos']
print(valores_unicos)

             Variable  Cantidad_Valores_Unicos
0              tmcode                    24149
1          clase_plan                        3
2             cfm_voz                     7747
3           cfm_datos                     1842
4          datos_plan                      170
5         recursos_gb                      170
6   unidad_homologada                        2
7          referencia                       31
8        departamento                       33
9           municipio                      974
10            cliente                     1604


## 8.1 - Análisis Exploratorio
### Cargamos las librerias necesasarias para el análisis exploratorio

In [82]:
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

### Total clientes por departamento y municipio

In [83]:
clientes_geo = df_merged.groupby(['departamento', 'municipio']).agg(total_clientes=('cliente', 'nunique')).reset_index()
print(clientes_geo)

     departamento       municipio  total_clientes
0        Amazonas     La Chorrera               1
1        Amazonas         Leticia              59
2        Amazonas   Puerto NariaO               9
3        Amazonas        Tarapaca               2
4       Antioquia       Abejorral              14
...           ...             ...             ...
1044       Vaupes         Taraira               1
1045      Vichada        Cumaribo              20
1046      Vichada    La Primavera              15
1047      Vichada  Puerto CarreaO              38
1048      Vichada   Santa Rosalia               5

[1049 rows x 3 columns]


### Promedio y distribución de CFM_VOZ y CFM_DATOS por clase de plan

In [84]:
cfm_stats = df_merged.groupby('clase_plan')[['cfm_voz', 'cfm_datos']].agg(['mean', 'std', 'min', 'max']).reset_index()
print(cfm_stats)

  clase_plan        cfm_voz                                        cfm_datos  \
                       mean           std    min           max          mean   
0    Abierto  122865.107198  2.206070e+06  350.0  2.055701e+08  26984.261821   
1      Mixto   77109.207481  6.165789e+04  350.0  8.446483e+05   7596.291281   
2    Prepago     350.000000  0.000000e+00  350.0  3.500000e+02   1800.000000   

                                  
            std     min      max  
0  26822.499245  1800.0  72398.0  
1  17132.476532  1800.0  72398.0  
2      0.000000  1800.0   1800.0  


### Clasificación de planes con datos ilimitados

In [85]:
df_merged['es_ilimitado'] = df_merged['datos_plan'].apply(lambda x: 1 if str(x).upper() == 'ILIM' else 0)
planes_ilimitados = df_merged[df_merged['es_ilimitado'] == 1]

## 8.2 – CLUSTERING KMeans

### Filtrar solo planes no ilimitados con datos numéricos válidos

In [86]:
cluster_df = df_merged[df_merged['es_ilimitado'] != 1][['cfm_voz', 'cfm_datos', 'recursos_gb']].dropna()
print(cluster_df)

         cfm_voz  cfm_datos  recursos_gb
0        6017.61   29471.27         55.0
1        6017.61   29471.27         55.0
2        6017.61   29471.27         55.0
3        6017.61   29471.27         55.0
4        6017.61   29471.27         55.0
...          ...        ...          ...
110644  12240.52   59853.69         37.0
110645  13462.01   65817.48         40.0
110646  13462.01   65817.48         40.0
110647  14516.56   70966.12         52.0
110648  16324.43   72398.00         72.0

[106208 rows x 3 columns]


### Escalamiento

In [87]:
scaler = StandardScaler()
cluster_scaled = scaler.fit_transform(cluster_df)
print(cluster_scaled)

[[-0.06029444  0.59460881  0.35570345]
 [-0.06029444  0.59460881  0.35570345]
 [-0.06029444  0.59460881  0.35570345]
 ...
 [-0.05572767  2.17252437  0.14163751]
 [-0.05508075  2.39604477  0.31289026]
 [-0.05397171  2.45820767  0.59831151]]


### KMeans con 3 clusters

In [88]:
kmeans = KMeans(n_clusters=3, random_state=0)
df_merged.loc[cluster_df.index, 'segmento_plan'] = kmeans.fit_predict(cluster_scaled)
print(df_merged["segmento_plan"])

0         2.0
1         2.0
2         2.0
3         2.0
4         2.0
         ... 
110644    2.0
110645    2.0
110646    2.0
110647    2.0
110648    2.0
Name: segmento_plan, Length: 110649, dtype: float64


## 8.3 – REGRESIÓN LINEAL


### El objetivo del modelo es predecir el valor esperado del cargo fijo mensual total (CFM_TOTAL) por cliente

#### Variable dependiente: CFM_TOTAL

In [89]:
df_merged['cfm_total'] = df_merged['cfm_voz'] + df_merged['cfm_datos']
print(df_merged['cfm_total'] )

0         35488.88
1         35488.88
2         35488.88
3         35488.88
4         35488.88
            ...   
110644    72094.21
110645    79279.49
110646    79279.49
110647    85482.68
110648    88722.43
Name: cfm_total, Length: 110649, dtype: float64


####  Asegurar los tipos de datos de las variables

In [90]:
df_merged['cliente'] = pd.to_numeric(df_merged['cliente'], errors='coerce').fillna(0)
df_merged['cliente'] = df_merged['cliente'].astype(str)

#### Agrupar por cliente único y calcular CFM_TOTAL promedio

In [97]:
df_cliente_agrupado = df_merged.groupby('cliente').agg({
    'cfm_total': 'mean',
    'clase_plan': lambda x: x.mode()[0],        # El tipo de plan más común por cliente
    'es_ilimitado': lambda x: x.mode()[0]       # Valor más frecuente (0 o 1)
}).reset_index()

#### Crear variables dummies

In [92]:
df_cliente_agrupado['clase_plan'] = df_cliente_agrupado['clase_plan'].astype(str)
df_model = pd.get_dummies(df_cliente_agrupado, columns=['clase_plan'], drop_first=True)

#### Separar variables predictoras y objetivo

In [93]:
X = df_model.drop(columns=['cliente', 'cfm_total'])
y = df_model['cfm_total']

#### Entrenamiento del modelo

In [94]:
reg = LinearRegression()
reg.fit(X, y)

LinearRegression()

#### Exportar los coeficientes

In [95]:
regression_summary = pd.DataFrame({
    'Variable': X.columns,
    'Coeficiente': reg.coef_
})
print(regression_summary)

           Variable   Coeficiente
0      es_ilimitado  26801.773725
1  clase_plan_Mixto  16195.049846


#### Calcular R² del modelo


In [96]:
r2 = reg.score(X, y)
print(f"R² del modelo: {r2:.4f}")

R² del modelo: 0.0451


### Interpretación de los resultados
#### Los planes con datos ilimitados tienden a tener un cfm_total promedio mayor en $26.801, respecto a los que no son ilimitados. Esto sugiere que estos planes no solo son más caros, sino que también están dirigidos a un segmento dispuesto a pagar más.

#### Los clientes con planes mixtos tienen un cfm_total promedio mayor en $16.195 respecto a los de planes base (posiblemente “Abierto”), lo que puede reflejar una combinación costo-valor percibido superior.

### Conclusión
#### El modelo de regresión lineal muestra que el CFM_TOTAL promedio por cliente aumenta significativamente cuando el plan es ilimitado o mixto. Esto indica que estas características son factores clave en la definición del valor comercial de un cliente. El modelo es útil para la planificación tarifaria, el diseño de planes personalizados y la proyección de ingresos en función del tipo de producto ofertado.